### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_df = pd.read_csv(school_data_to_load)
student_df = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
merged_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])

## District Summary

In [3]:
# Tally up information for District Summanry

total_schools = school_df['school_name'].count()
total_students = student_df['student_name'].count()
total_budget = school_df['budget'].sum()
av_math_score = student_df['math_score'].mean()
av_read_score = student_df['reading_score'].mean()
math_pass_count = len(student_df.loc[(student_df["math_score"] >= 70)])
read_pass_count = len(student_df.loc[(student_df["reading_score"] >= 70)])
per_pass_math = (math_pass_count / total_students) * 100
per_pass_read = (read_pass_count / total_students) * 100
overall_pass_rate = ((per_pass_math + per_pass_read) / 2)

district_df = pd.DataFrame({"Total Schools": [total_schools],
                           "Total Students": [total_students],
                           "Total Budget": [total_budget],
                           "Average Math Score": [av_math_score],
                           "Average Reading Score": [av_read_score],
                           "% Passing Math": [per_pass_math],
                           "% Passing Reading": [per_pass_read],
                           "% Overall Passing": overall_pass_rate})

# Changing format via mapping
district_df['Total Students'] = district_df['Total Students'].map("{:,}".format)
district_df['Total Budget'] = district_df['Total Budget'].map("${:,.2f}".format)

# Show district summanry

district_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

In [37]:
def math_check_pass(row):
    if row['math_score'] >= 70:
        return 1
    else:
        return 0

def read_check_pass(row):
    if row['reading_score'] >= 70:
        return 1
    else:
        return 0

merged_df["Reading Pass Count"] = merged_df.apply(read_check_pass, axis=1)
merged_df["Math Pass Count"] = merged_df.apply(math_check_pass, axis =1)

def per_student_budget(row):
    return row['budget'] / row['size']

def per_passing_math(row):
    return 

merged_df["Per Student Budget"] = merged_df.apply(per_student_budget, axis=1)

grouped =merged_df.groupby('Math Pass Count')

aggregation = {
    'School Type': ('type', max),
    'Total Students': ('size', "count"),
    'Total School Budget': ('budget', max),
    'Per Student Budget': ('Per Student Budget', max),
    'Average Math Score': ('math_score', "mean"),
    'Average Reading Score': ('reading_score', "mean"),
    'Math Pass Count': ('Math Pass Count', "sum"),
    'Read Pass Count': ('Reading Pass Count', "sum")}
    
preschool_summary_df = merged_df.groupby('school_name').agg(**aggregation)   

preschool_summary_df['% Passing Math'] = preschool_summary_df['Math Pass Count'] / preschool_summary_df['Total Students'] * 100
preschool_summary_df['% Passing Reading'] = preschool_summary_df['Read Pass Count'] / preschool_summary_df['Total Students'] * 100
preschool_summary_df['% Overall Passing Rate'] = (preschool_summary_df['% Passing Math'] + preschool_summary_df['% Passing Reading']) / 2

# Added for later calculations that need budget as a float
School_summary2_df = preschool_summary_df[["School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]


# Changing format via mapping
preschool_summary_df['Per Student Budget'] = preschool_summary_df['Per Student Budget'].map("${:,.2f}".format)
preschool_summary_df['Total School Budget'] = preschool_summary_df['Total School Budget'].map("${:,.2f}".format)
School_summary_df = preschool_summary_df[["School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [38]:
SchSumSortGood_df = School_summary_df.sort_values(by='% Overall Passing Rate', ascending=False)
SchSumSortGood_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [39]:
SchSumSortBad_df = School_summary_df.sort_values(by='% Overall Passing Rate', ascending=False)
SchSumSortBad_df.tail()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323


## Math Scores by Grade

In [40]:
G_9th_s = merged_df.loc[merged_df.grade=='9th'].groupby(['school_name'])['math_score'].mean().reset_index()
G_9th_s.rename(columns = {'math_score':'9th'}, inplace = True)
G_10th_s = merged_df.loc[merged_df.grade=='10th'].groupby(['school_name'])['math_score'].mean().reset_index()
G_10th_s.rename(columns = {'math_score':'10th'}, inplace = True)
G_11th_s = merged_df.loc[merged_df.grade=='11th'].groupby(['school_name'])['math_score'].mean().reset_index()
G_11th_s.rename(columns = {'math_score':'11th'}, inplace = True)
G_12th_s = merged_df.loc[merged_df.grade=='12th'].groupby(['school_name'])['math_score'].mean().reset_index()
G_12th_s.rename(columns = {'math_score':'12th'}, inplace = True)
math_merge_df = pd.merge( G_9th_s,G_10th_s, on='school_name')
math_merge_df = pd.merge( math_merge_df,G_11th_s, on='school_name')
math_merge_df = pd.merge( math_merge_df,G_12th_s, on='school_name')
math_merge_df

,school_name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [41]:
G_9th_r = merged_df.loc[merged_df.grade=='9th'].groupby(['school_name'])['reading_score'].mean().reset_index()
G_9th_r.rename(columns = {'reading_score':'9th'}, inplace = True)
G_10th_r = merged_df.loc[merged_df.grade=='10th'].groupby(['school_name'])['reading_score'].mean().reset_index()
G_10th_r.rename(columns = {'reading_score':'10th'}, inplace = True)
G_11th_r = merged_df.loc[merged_df.grade=='11th'].groupby(['school_name'])['reading_score'].mean().reset_index()
G_11th_r.rename(columns = {'reading_score':'11th'}, inplace = True)
G_12th_r = merged_df.loc[merged_df.grade=='12th'].groupby(['school_name'])['reading_score'].mean().reset_index()
G_12th_r.rename(columns = {'reading_score':'12th'}, inplace = True)
read_merge_df = pd.merge( G_9th_r,G_10th_r, on='school_name')
read_merge_df = pd.merge( read_merge_df,G_11th_r, on='school_name')
read_merge_df = pd.merge( read_merge_df,G_12th_r, on='school_name')
read_merge_df

,school_name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [56]:
School_spending_df = School_summary2_df[['Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']].reset_index()

In [57]:
School_spending_df

,school_name,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Bailey High School,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Hernandez High School,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
6,Holden High School,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
7,Huang High School,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
9,Pena High School,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


In [58]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["$0-$585", "$585-615", "$615-645", "$645-675"]

spending_summary_df['School_Spending_Summary'] = pd.cut(School_spending_df["Per Student Budget"], spending_bins, labels=group_names)
spending_summary_df

0                                                                   $615-645
1                                                                    $0-$585
2                                                                   $615-645
3                                                                   $615-645
4                                                                   $615-645
5                                                                   $645-675
6                                                                    $0-$585
7                                                                   $645-675
8                                                                   $645-675
9                                                                   $585-615
10                                                                  $615-645
11                                                                  $585-615
12                                                                  $615-645

In [17]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
